In [1]:
import pandas as pd
import json
import re
from sklearn.cluster import DBSCAN
from sentence_transformers import SentenceTransformer
import umap
import numpy as np
import joblib

/opt/anaconda3/envs/alerts/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datasets import load_dataset
import faiss

In [3]:
SBERT_MODEL = "all-MiniLM-L6-v2"


## Functions

In [4]:
def load_vectorizer(model=SBERT_MODEL):
     return SentenceTransformer(model)
    
def vectorize_text(model,texts):
    
    return model.encode(texts, show_progress_bar=True)

In [5]:
def umap_embed(embeddings,model_name="umap-model",n_neighbors=15,n_components=50,min_dist=0.05,train=False):
    
    if train:
        umap_embeddings = umap.UMAP(n_neighbors=n_neighbors, 
                                    n_components=n_components, 
                                    min_dist = min_dist,
                                    metric='cosine',
                                    ).fit(embeddings)
        joblib.dump(umap_embeddings,f"umap-{model_name}.pkl")
        print("Saved Umap model.")

    
    umap_embeddings = joblib.load(f"umap-{model_name}.pkl")
    embeddings = umap_embeddings.transform(embeddings)
    
    return embeddings

In [6]:
def get_data_by_indices(dataset, indices):
    
    return dataset.select(indices).to_pandas()

## Data-prep

In [7]:
model = load_vectorizer()

In [8]:
dataset = load_dataset("ehartford/dolphin")
#.shuffle().sample(range(0,10000))

Found cached dataset json (/Users/shahules/.cache/huggingface/datasets/ehartford___json/ehartford--dolphin-a93dfe869bfca67a/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)
100%|█████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.70it/s]


In [9]:
train_dataset = dataset['train'].shuffle().select(range(0,1000))
input_text = [item["input"].strip() for item in train_dataset]

In [10]:
input_vectors = vectorize_text(model, input_text)

Batches: 100%|██████████████████████████████████████████| 32/32 [01:45<00:00,  3.30s/it]


In [11]:
# input_vectors_umap = umap_embed(input_vectors,train=True)

In [11]:
d = input_vectors.shape[-1]

In [12]:
quantizer = faiss.IndexFlatIP(d)
index = faiss.IndexIVFFlat(quantizer,d,5)
index.is_trained

False

In [ ]:
index.train(input_vectors[:100])


In [ ]:
index.add(input_vectors)

In [ ]:
I,D = index.search(embed_vectors[35].reshape(1,-1),k)
